In [85]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import time
options = webdriver.ChromeOptions()
options.add_argument("--incognito")

# Install and setup Chrome driver
chrome_driver_path = ChromeDriverManager().install()
service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=service, options=options)
driver.get("https://montreal.ca/role-evaluation-fonciere")
# Maximize window and set implicit wait
driver.maximize_window()

In [86]:
# Locate the div containing the radio button
div_element = driver.find_element(By.CSS_SELECTOR, "div[data-test='item-lot-renove']")

# Click the div
div_element.click()

In [87]:
# Scroll down in steps before clicking
for _ in range(3):  # Adjust the range if needed
    driver.find_element(By.TAG_NAME, "body").send_keys(Keys.PAGE_DOWN)
    time.sleep(0.5)

# Click the button

time.sleep(0.5)
# Locate and click the button
button = driver.find_element(By.CSS_SELECTOR, "button[data-test='submit']")
button.click()

In [88]:
lot_field = driver.find_element(By.CSS_SELECTOR, "input[class='form-control']")
lot_field.clear()
lot = "4254908"
lot_field.send_keys(lot)
# button = driver.find_element(By.CSS_SELECTOR, "button[class='btn-squared btn btn-primary']")


In [89]:
button = driver.find_element(By.CSS_SELECTOR, "button[class='btn-squared btn btn-primary']")
button.click()

In [90]:
items = driver.find_elements(By.CSS_SELECTOR, "button[data-test='button']")
len(items)

1

In [91]:
item =  driver.find_element(By.CSS_SELECTOR, "button[data-test='button']")
item.click()

In [92]:
def get_text_safe(element):
    try:
        return element.text.split("\n", 1)[-1].strip() if "\n" in element.text else element.text.strip()
    except AttributeError:
        return ""

# Function to extract data from a section
def extract_section_data(section):
    data = {}
    items = section.find_elements(By.CSS_SELECTOR, "li.list-item")
    for item in items:
        label = get_text_safe(item.find_element(By.CSS_SELECTOR, ".list-item-label"))
        value = get_text_safe(item.find_element(By.CSS_SELECTOR, ".list-item-content"))
        data[label] = value
    return data

identification_section = driver.find_element(By.CSS_SELECTOR, "section.layouts\\:stack h2#identification + ul")
identification_data = extract_section_data(identification_section)

print("1- Identification de l'unité d'évaluation:")
for key, value in identification_data.items():
    print(f"{key}: {value}")



# Extract data from the "Propriétaire" section
proprietaires_section = driver.find_element(By.CSS_SELECTOR, "section.layouts\\:stack h2#proprietaires + ul")
proprietaires_data = extract_section_data(proprietaires_section)
print("\n2- Propriétaire:")
for key, value in proprietaires_data.items():
    print(f"{key}: {value}")



terrain_section_script = """
var h3Elements = document.querySelectorAll('h3.h4');
for (var i = 0; i < h3Elements.length; i++) {
    if (h3Elements[i].textContent.trim() === 'Caractéristiques du terrain') {
        return h3Elements[i].nextElementSibling;
    }
}
return null;
"""

# Extract "Caractéristiques du terrain"
terrain_section = driver.execute_script(terrain_section_script)
terrain_data = extract_section_data(terrain_section)

batiment_section_script = """
var h3Elements = document.querySelectorAll('h3.h4');
for (var i = 0; i < h3Elements.length; i++) {
    if (h3Elements[i].textContent.trim() === 'Caractéristiques du bâtiment principal') {
        return h3Elements[i].nextElementSibling;
    }
}
return null;
"""
# Extract "Caractéristiques du bâtiment principal"
batiment_section = driver.execute_script(batiment_section_script)
batiment_data = extract_section_data(batiment_section)

print("\n3.1 Caractéristiques du terrain:")
for key, value in terrain_data.items():
    print(f"{key}: {value}")

print("\n3.2 Caractéristiques du bâtiment principal:")
for key, value in batiment_data.items():
    print(f"{key}: {value}")

# ============================================================
# Extract data from the "Valeurs au rôle d’évaluation" section
# ============================================================

# Use JavaScript to find the <h3> element with the text "Rôle courant"
Rôle_courant_script = """
var h3Elements = document.querySelectorAll('h3.h4');
for (var i = 0; i < h3Elements.length; i++) {
    if (h3Elements[i].textContent.trim() === 'Rôle courant') {
        return h3Elements[i].nextElementSibling;
    }
}
return null;
"""
role_courant_section = driver.execute_script(Rôle_courant_script)
role_courant_data =  extract_section_data(role_courant_section)
print("\nValeurs au rôle d’évaluation - Rôle courant:")


Rôle_antérieur_script = """
var h3Elements = document.querySelectorAll('h3.h4');
for (var i = 0; i < h3Elements.length; i++) {
    if (h3Elements[i].textContent.trim() === 'Rôle antérieur') {
        return h3Elements[i].nextElementSibling;
    }
}
return null;
"""
role_courant_section = driver.execute_script(Rôle_antérieur_script)
role_anterieur_data =  extract_section_data(role_courant_section)

print("\n4.1 Valeurs au rôle d’évaluation - Rôle courant:")
for key, value in role_courant_data.items():
    print(f"{key}: {value}")

print("\n4.2 Valeurs au rôle d’évaluation - Rôle antérieur:")
for key, value in role_anterieur_data.items():
    print(f"{key}: {value}")

# ============================================================
# Extract data from the "Répartition fiscale" section
# ============================================================
repartition_section = driver.find_element(By.CSS_SELECTOR, "section.layouts\\:stack h2#repartition + ul")
repartition_data = extract_section_data(repartition_section)

print("\n5 Répartition fiscale:")
for key, value in repartition_data.items():
    print(f"{key}: {value}")

1- Identification de l'unité d'évaluation:
Adresse: 130 Avenue Monterrey
Arrondissement: Arrondissement
Numéro de lot: 4254908
Numéro de matricule: 8134-19-9885-4-000-0000
Utilisation prédominante: Logement
Numéro d'unité de voisinage: 614
Numéro de compte foncier: 20 - F00235400

2- Propriétaire:
Nom: MYLES, SAMUEL
Statut aux fins d'imposition scolaire: Personne physique
Adresse postale: 130 AV MONTERREY , POINTE-CLAIRE QUEBEC, H9R 3W4
Date d'inscription au rôle: 2022-06-15

3.1 Caractéristiques du terrain:
Mesure frontale: 30,48 m
Superficie: 687,5 m2

3.2 Caractéristiques du bâtiment principal:
Nombre d'étages: 1
Année de construction: 1954
Aire d'étages: 94,5 m2
Genre de construction: De plain-pied
Lien physique: Détaché
Nombre de logements: 1
Nombre de locaux non résidentiels: Nombre de locaux non résidentiels
Nombre de chambres locatives: Nombre de chambres locatives

Valeurs au rôle d’évaluation - Rôle courant:

4.1 Valeurs au rôle d’évaluation - Rôle courant:
Date de référence 

In [ ]:
import csv
all_data = {
    "ID" : lot,
    "Identification": identification_data,
    "Propriétaire": proprietaires_data,
    "Caractéristiques du terrain": terrain_data,
    "Caractéristiques du bâtiment principal": batiment_data,
    "Rôle courant": role_courant_data,
    "Rôle antérieur": role_anterieur_data,
    "Répartition fiscale": repartition_data,
}

# Save data to CSV
csv_file = "extracted_data.csv"
with open(csv_file, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    
    # Write headers
    headers = ["Section", "Key", "Value"]
    writer.writerow(headers)
    
    # Write rows
    for section_name, section_data in all_data.items():
        for key, value in section_data.items():
            writer.writerow([section_name, key, value])

print(f"Data saved to {csv_file}")


Data saved to extracted_data.csv


In [174]:
import csv
import os

# Example data
all_data = {
    "ID": lot,  # Not necessarily unique
    "Identification": identification_data,
    "Propriétaire": proprietaires_data,
    "Caractéristiques du terrain": terrain_data,
    "Caractéristiques du bâtiment principal": batiment_data,
    "Rôle courant": role_courant_data,
    "Rôle antérieur": role_anterieur_data,
    "Répartition fiscale": repartition_data,
}

# Define the CSV file
csv_file = "extracted_data1.csv"

# Check if the file exists
file_exists = os.path.isfile(csv_file)

# Prepare the new row
new_row = {
    **{"ID": all_data["ID"]},  # Add the ID (not necessarily unique)
    **all_data["Identification"],
    **all_data["Propriétaire"],
    **all_data["Caractéristiques du terrain"],
    **all_data["Caractéristiques du bâtiment principal"],
    **all_data["Rôle courant"],
    **all_data["Rôle antérieur"],
    **all_data["Répartition fiscale"],
}

# Append the new row to the CSV file
with open(csv_file, mode="a", newline="", encoding="utf-8") as file:
    writer = csv.DictWriter(file, fieldnames=new_row.keys())
    
    # Write headers only if the file is new
    if not file_exists:
        writer.writeheader()
    
    # Write the new row
    writer.writerow(new_row)

print(f"Data appended to {csv_file}")

Data appended to extracted_data1.csv


In [63]:
Les_informations_script = """
var elements = document.querySelectorAll('div.list-item-content');
for (var i = 0; i < elements.length; i++) {
    var label = elements[i].querySelector('div.list-item-label');
    if (label && label.textContent.trim() === 'Les informations présentées dans ce rapport sont en date du') {
        return elements[i]; // Return the parent div containing the label and date
    }
}
return null;
"""

Les_informations = driver.execute_script(Les_informations_script)
Les_informations_data =  extract_section_data(Les_informations)

#print("\n6 Les_informations_data:")
#for key, value in Les_informations_data.items():
   # print(f"{key}: {value}")
print(Les_informations.text)

Date_du_script = """
var elements = document.querySelectorAll('div.list-item-content');
for (var i = 0; i < elements.length; i++) {
    var label = elements[i].querySelector('div.list-item-label');
    if (label && label.textContent.trim() === 'Date du rapport') {
        return elements[i]; // Return the parent div containing the label and date
    }
}
return null;
"""

Date_du = driver.execute_script(Date_du_script)
Date_du_data = extract_section_data(Date_du)

#print("\n7 Date_du:")
#for key, value in Date_du_data.items():
    #print(f"{key}: {value}")
print(Date_du.text)

Les informations présentées dans ce rapport sont en date du
2025-03-13
Date du rapport
2025-03-21


In [52]:
try:
    # Find all divs with class 'list-item-content'
    elements = driver.find_elements(By.CSS_SELECTOR, 'div.list-item-content')
    Les_informations = None

    # Loop through the elements to find the one with the desired label
    for element in elements:
        label = element.find_element(By.CSS_SELECTOR, 'div.list-item-label').text
        if label.strip() == 'Les informations présentées dans ce rapport sont en date du':
            Les_informations = element
            break

    # Extract and print data
    if Les_informations:
        Les_informations_data = extract_section_data(Les_informations)
        print("\n6 Les_informations_data:")
        for key, value in Les_informations_data.items():
            print(f"{key}: {value}")
    else:
        print("No matching element found.")
except Exception as e:
    print("An error occurred:", e)
print(Les_informations.text)


6 Les_informations_data:
Les informations présentées dans ce rapport sont en date du
2025-03-13
